In [6]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.model_selection import cross_validate
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder

# carga fichero

In [10]:
try:
    df_trabajo_fusion = pd.read_csv("../output/Datos_Fusionadosv1.csv")

except Exception as e:
    print(f"Error al cargar los datos: {str(e)}")

In [11]:
df_trabajo_fusion.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9951 entries, 0 to 9950
Data columns (total 20 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   ID_Orden                                9951 non-null   int64  
 1   ID_Equipo                               9951 non-null   int64  
 2   Fecha                                   9951 non-null   object 
 3   Tipo_Mantenimiento                      9951 non-null   object 
 4   Costo_Mantenimiento                     9951 non-null   float64
 5   Duracion_Horas                          9951 non-null   int64  
 6   Ubicacion                               9951 non-null   object 
 7   Frecuencia de mantenimiento correctivo  9951 non-null   int64  
 8   ID_Registro                             9951 non-null   int64  
 9   Temperatura_C                           9951 non-null   float64
 10  Vibracion_mm_s                          9951 non-null   floa

### Eliminar columnas irrelevantes

In [13]:
# Eliminar columnas irrelevantes (iniciales hay que añadir mas)

discarded_columns = ["Modelo", "Fabricante", "ID_Registro", "Ubicacion"]
df_trabajo_fusion_limpio = df_trabajo_fusion.drop(discarded_columns, axis=1)
df_trabajo_fusion_limpio.head()

,ID_Orden,ID_Equipo,Fecha,Tipo_Mantenimiento,Costo_Mantenimiento,Duracion_Horas,Frecuencia de mantenimiento correctivo,Temperatura_C,Vibracion_mm_s,Horas_Operativas,Mes,Horas_Operativas_mean,Tipo_Equipo,Potencia_kW,Horas_Recomendadas_Revision,Fecha_Revision
0,1,36,2020-01-01 00:00:00,Preventivo,6754.12,3,9,139.97,4.31,41114.0,2020-01,41114.0,Motor,4351.0,3549,2020-01-01 00:00:00
1,2,244,2020-01-01 01:00:00,Preventivo,4274.63,35,11,141.25,4.82,93042.0,2020-02,93042.0,Generador,1206.0,1061,2020-02-14 06:00:00
2,3,297,2020-01-01 02:00:00,Preventivo,5752.00,25,9,71.11,3.24,42929.0,2020-01,53789.6,Transformador,3728.0,8016,2020-11-30 02:00:00
3,4,431,2020-01-01 03:00:00,Correctivo,5690.14,8,14,89.23,4.12,81714.0,2020-01,81714.0,Transformador,253.0,7764,2020-11-30 02:00:00
4,5,403,2020-01-01 04:00:00,Correctivo,7048.79,28,8,61.95,4.66,96881.0,2020-01,72512.0,Motor,2597.0,4993,2020-11-30 02:00:00


# Conversion datos categoricos

In [ ]:
# Extraer columnas numéricas y categóricas
from sklearn.compose import make_column_selector as selector

numerical_columns_selector = selector(dtype_exclude=object)
categorical_columns_selector = selector(dtype_include=object)

numerical_columns = numerical_columns_selector(df_trabajo_fusion_limpio)
categorical_columns = categorical_columns_selector(df_trabajo_fusion_limpio)

print("Numerical columns", numerical_columns)
print("Categorical columns", categorical_columns)

In [ ]:
# Crear el LabelEncoder
label_encoders = {}

# Ajustar y transformar las columnas categóricas
for column in categorical_columns:
    le = LabelEncoder()
    df_trabajo_fusion_limpio[column] = le.fit_transform(df_trabajo_fusion_limpio[column])
    label_encoders[column] = le

print(df_trabajo_fusion_limpio.head())